In [1]:
import os
import time

import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
from torch.utils.data import Dataset

from torchvision import transforms

import matplotlib.pyplot as plt
from PIL import Image

if torch.cuda.is_available():
    torch.backends.cudnn.deterministic = True

In [2]:
##########################
### SETTINGS
##########################

# Hyperparameters
RANDOM_SEED = 123
LEARNING_RATE = 0.0005
BATCH_SIZE = 64
NUM_EPOCHS = 50

# Architecture
NUM_FEATURES = 28*28
NUM_CLASSES = 345

# Other
DEVICE = "cuda:0"
GRAYSCALE = True

In [3]:
# Load data
images = np.load("Data/1000/sample_1000_image.npy")
labels = np.load("Data/1000/sample_1000_label.npy")

# Normalize image data.  0-255 to 0-1
images = images / 255
df = pd.DataFrame(np.concatenate((images, labels), axis=1))

# Rename the last column as "label"
df.rename(columns={784:"label"}, inplace=True)

# Convert label column to integer type
df['label'] = df['label'].astype('int64')

In [4]:
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

In [5]:
# Get "img" data frame and "lbl" series from the df
img = df.iloc[:, 0:-1]
lbl = df['label']
# Split into train, validation and test set
x_train1, x_test, y_train1, y_test = train_test_split(img, lbl, test_size = 0.10, random_state = 123, stratify = lbl)
x_train, x_valid, y_train, y_valid = train_test_split(x_train1, y_train1, test_size = 0.2, random_state = 123, stratify = y_train1)

In [6]:
print(x_train.shape)
print(x_valid.shape)
print(x_test.shape)
print(y_train.shape)
print(y_valid.shape)
print(y_test.shape)

(248400, 784)
(62100, 784)
(34500, 784)
(248400,)
(62100,)
(34500,)


In [7]:
# Convert dataframe to tensor
x_train = torch.tensor(x_train.values)
y_train = torch.tensor(y_train.values)

x_valid = torch.tensor(x_valid.values)
y_valid = torch.tensor(y_valid.values)

x_test = torch.tensor(x_test.values)
y_test = torch.tensor(y_test.values)

In [8]:
# Reshape all tensors to the size of (length, 1, 28, 28)
x_train = x_train.reshape((-1, 1, 28, 28))
x_valid = x_valid.reshape((-1, 1, 28, 28))
x_test = x_test.reshape((-1, 1, 28, 28))

In [9]:
# Create dataloaders

train_dataset = TensorDataset(x_train, y_train)
valid_dataset = TensorDataset(x_valid, y_valid)
test_dataset = TensorDataset(x_test, y_test)

train_loader = DataLoader(dataset=train_dataset, 
                          batch_size=BATCH_SIZE, 
                          shuffle=True, num_workers=4)

valid_loader = DataLoader(dataset=valid_dataset, 
                         batch_size=BATCH_SIZE, 
                         shuffle=False, num_workers=4)

test_loader = DataLoader(dataset=test_dataset, 
                         batch_size=BATCH_SIZE, 
                         shuffle=False, num_workers=4)

In [10]:
# Checking the dataset
for images, labels in train_loader:  
    print('Image batch dimensions:', images.shape)
    print('Image label dimensions:', labels.shape)
    break

Image batch dimensions: torch.Size([64, 1, 28, 28])
Image label dimensions: torch.Size([64])


In [11]:
device = torch.device(DEVICE if torch.cuda.is_available() else "cpu")
torch.manual_seed(0)

num_epochs = 2
for epoch in range(num_epochs):

    for batch_idx, (x, y) in enumerate(train_loader):
        
        print('Epoch:', epoch+1, end='')
        print(' | Batch index:', batch_idx, end='')
        print(' | Batch size:', y.size()[0])
        
        x = x.to(device)
        y = y.to(device)
        break

Epoch: 1 | Batch index: 0 | Batch size: 64
Epoch: 2 | Batch index: 0 | Batch size: 64


In [12]:

##########################
### MODEL
##########################


class LeNet5(nn.Module):

    def __init__(self, num_classes, grayscale=False):
        super(LeNet5, self).__init__()
        
        self.grayscale = grayscale
        self.num_classes = num_classes

        if self.grayscale:
            in_channels = 1
        else:
            in_channels = 3

        self.features = nn.Sequential(
            
            nn.Conv2d(in_channels, 6, kernel_size=5),
            nn.MaxPool2d(kernel_size=2),
            nn.Conv2d(6, 16, kernel_size=5),
            nn.MaxPool2d(kernel_size=2)
        )

        self.classifier = nn.Sequential(
            nn.Linear(16*4*4, 120),
            nn.Linear(120, 84),
            nn.Linear(84, num_classes),
        )


    def forward(self, x):
        x = self.features(x)
        x = torch.flatten(x, 1)
        logits = self.classifier(x)
        probas = F.softmax(logits, dim=1)
        return logits, probas

In [13]:
torch.manual_seed(RANDOM_SEED)

model = LeNet5(NUM_CLASSES, GRAYSCALE)
model = model.to(DEVICE)

optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

In [31]:
DEVICE

'cuda:0'

In [14]:
def print_sizes(self, input, output):

    print('Inside ' + self.__class__.__name__ + ' forward')
    print('input size:', input[0].size())
    print('output size:', output.data.size())

    
## Debugging


# model.features[0].register_forward_hook(print_sizes)
# model.features[1].register_forward_hook(print_sizes)
# model.features[2].register_forward_hook(print_sizes)
# model.features[3].register_forward_hook(print_sizes)

# model.classifier[0].register_forward_hook(print_sizes)
# model.classifier[1].register_forward_hook(print_sizes)
# model.classifier[2].register_forward_hook(print_sizes)


In [15]:
def compute_accuracy(model, data_loader, device):
    correct_pred, num_examples = 0, 0
    for i, (features, targets) in enumerate(data_loader):
        
        features = features.float()
        features = features.to(device)
        targets = targets.to(device)

        logits, probas = model(features)
        _, predicted_labels = torch.max(probas, 1)
        num_examples += targets.size(0)
        correct_pred += (predicted_labels == targets).sum()
    return correct_pred.float()/num_examples * 100

In [16]:
start_time = time.time()

for epoch in range(NUM_EPOCHS):
    
    model.train()
    for batch_idx, (features, targets) in enumerate(train_loader):
        
        features = features.float()
        features = features.to(DEVICE)
        targets = targets.to(DEVICE)
            
        ### FORWARD AND BACK PROP
        logits, probas = model(features)
        cost = F.cross_entropy(logits, targets)
        optimizer.zero_grad()
        
        cost.backward()
        
        ### UPDATE MODEL PARAMETERS
        optimizer.step()
        
        ### LOGGING
        if not batch_idx % 500:
            print ('Epoch: %03d/%03d | Batch %04d/%04d | Cost: %.4f' 
                   %(epoch+1, NUM_EPOCHS, batch_idx, 
                     len(train_loader), cost))


    model.eval()
    with torch.set_grad_enabled(False): # save memory during inference
        print('Epoch: %03d/%03d | Train: %.3f%% | Validation: %.3f%%' % (
              epoch+1, NUM_EPOCHS, 
              compute_accuracy(model, train_loader, device=DEVICE),
              compute_accuracy(model, valid_loader, device=DEVICE) ))
        
    print('Time elapsed: %.2f min' % ((time.time() - start_time)/60))
    
print('Total Training Time: %.2f min' % ((time.time() - start_time)/60))

Epoch: 001/050 | Batch 0000/3882 | Cost: 5.8429
Epoch: 001/050 | Batch 0500/3882 | Cost: 4.3395
Epoch: 001/050 | Batch 1000/3882 | Cost: 4.2463
Epoch: 001/050 | Batch 1500/3882 | Cost: 3.5139
Epoch: 001/050 | Batch 2000/3882 | Cost: 3.6813
Epoch: 001/050 | Batch 2500/3882 | Cost: 3.8267
Epoch: 001/050 | Batch 3000/3882 | Cost: 3.2623
Epoch: 001/050 | Batch 3500/3882 | Cost: 3.5534
Epoch: 001/050 | Train: 31.184% | Validation: 30.712%
Time elapsed: 0.50 min
Epoch: 002/050 | Batch 0000/3882 | Cost: 3.0288
Epoch: 002/050 | Batch 0500/3882 | Cost: 3.2172
Epoch: 002/050 | Batch 1000/3882 | Cost: 2.8994
Epoch: 002/050 | Batch 1500/3882 | Cost: 3.4659
Epoch: 002/050 | Batch 2000/3882 | Cost: 3.3334
Epoch: 002/050 | Batch 2500/3882 | Cost: 2.9032
Epoch: 002/050 | Batch 3000/3882 | Cost: 3.0670
Epoch: 002/050 | Batch 3500/3882 | Cost: 3.1190
Epoch: 002/050 | Train: 36.081% | Validation: 35.438%
Time elapsed: 0.98 min
Epoch: 003/050 | Batch 0000/3882 | Cost: 2.4372
Epoch: 003/050 | Batch 0500/38

Epoch: 018/050 | Train: 47.760% | Validation: 45.412%
Time elapsed: 8.81 min
Epoch: 019/050 | Batch 0000/3882 | Cost: 1.8553
Epoch: 019/050 | Batch 0500/3882 | Cost: 2.7019
Epoch: 019/050 | Batch 1000/3882 | Cost: 2.0054
Epoch: 019/050 | Batch 1500/3882 | Cost: 3.0068
Epoch: 019/050 | Batch 2000/3882 | Cost: 2.3781
Epoch: 019/050 | Batch 2500/3882 | Cost: 2.3243
Epoch: 019/050 | Batch 3000/3882 | Cost: 2.0548
Epoch: 019/050 | Batch 3500/3882 | Cost: 2.3907
Epoch: 019/050 | Train: 47.925% | Validation: 45.596%
Time elapsed: 9.29 min
Epoch: 020/050 | Batch 0000/3882 | Cost: 2.5784
Epoch: 020/050 | Batch 0500/3882 | Cost: 2.3957
Epoch: 020/050 | Batch 1000/3882 | Cost: 2.5554
Epoch: 020/050 | Batch 1500/3882 | Cost: 2.5139
Epoch: 020/050 | Batch 2000/3882 | Cost: 2.2983
Epoch: 020/050 | Batch 2500/3882 | Cost: 2.8447
Epoch: 020/050 | Batch 3000/3882 | Cost: 2.6835
Epoch: 020/050 | Batch 3500/3882 | Cost: 2.5994
Epoch: 020/050 | Train: 48.081% | Validation: 45.647%
Time elapsed: 9.78 min
E

Epoch: 036/050 | Batch 3000/3882 | Cost: 2.4265
Epoch: 036/050 | Batch 3500/3882 | Cost: 2.2319
Epoch: 036/050 | Train: 49.727% | Validation: 46.969%
Time elapsed: 17.88 min
Epoch: 037/050 | Batch 0000/3882 | Cost: 2.0551
Epoch: 037/050 | Batch 0500/3882 | Cost: 2.0063
Epoch: 037/050 | Batch 1000/3882 | Cost: 2.2149
Epoch: 037/050 | Batch 1500/3882 | Cost: 1.7955
Epoch: 037/050 | Batch 2000/3882 | Cost: 2.1241
Epoch: 037/050 | Batch 2500/3882 | Cost: 1.9355
Epoch: 037/050 | Batch 3000/3882 | Cost: 2.8716
Epoch: 037/050 | Batch 3500/3882 | Cost: 1.8470
Epoch: 037/050 | Train: 49.334% | Validation: 46.631%
Time elapsed: 18.38 min
Epoch: 038/050 | Batch 0000/3882 | Cost: 2.1237
Epoch: 038/050 | Batch 0500/3882 | Cost: 2.3148
Epoch: 038/050 | Batch 1000/3882 | Cost: 2.5067
Epoch: 038/050 | Batch 1500/3882 | Cost: 2.3305
Epoch: 038/050 | Batch 2000/3882 | Cost: 2.5184
Epoch: 038/050 | Batch 2500/3882 | Cost: 2.3125
Epoch: 038/050 | Batch 3000/3882 | Cost: 2.2501
Epoch: 038/050 | Batch 3500/

In [17]:

with torch.set_grad_enabled(False): # save memory during inference
    print('Test accuracy: %.2f%%' % (compute_accuracy(model, test_loader, device=DEVICE)))

Test accuracy: 46.94%
